In [1]:
using DifferentialEquations
using Plots
using Quadrature
using Roots
using DelimitedFiles

In [2]:
function piece_wise_linear(x_interval, y_interval, x) #these are the intervals in the x and y, each interval will have a lin fuunction
    if x < x_interval[1,1]
        my_i = 1;
    end
    if x >= x_interval[end, 2]
        my_i = size(x_interval,1);
    end
    for i = 1:size(x_interval, 1)
        if x_interval[i,1] <= x && x < x_interval[i, 2]
            my_i = i;
        end
    end
    a = (x - x_interval[my_i, 1])/(x_interval[my_i,2] - x_interval[my_i,1]);
    return a * y_interval[my_i,2] + (1 - a) * y_interval[my_i,1];
end

piece_wise_linear (generic function with 1 method)

In [3]:
function ideal_gas_der(ρ, par)
    k,γ = par
    return k*γ*ρ^(γ - 1)
end
function piece_wise_linear_csound(ρ, par)
    return piece_wise_linear(par[1], par[2], ρ) #par is an array with x_int and y_int
end
function integrate_EOS(ρ, der, par) #pressure comes from here
    prob = QuadratureProblem(der, 0.0, ρ, par)
    sol = solve(prob, QuadGKJL())#,reltol = 1e-10)
    return sol[1]
end

integrate_EOS (generic function with 1 method)

In [4]:
function make_TOV(ρ₀, dP, par)
    function TOV_equations!(du, u, par, r)
        myRho = u[1]
        myM = u[2]
        myP = u[3] 
        mydP = dP(myRho, par) #equal to dp/dρ

        if r <= 0.0
            du[1] = 0.0
            du[2] = 0.0
            du[3] = 0.0
        else
            du[3] = -((myM + 4π*(r^3)*myP)*(myRho + myP)) / (r*(r - 2*myM)) #this is pressure
            du[2] = (4π*r^2)*myRho #this is mass
            du[1] = du[3]/mydP #this is density
        end
    end
    u0 = [ρ₀, 0.0, integrate_EOS(ρ₀, dP, par)];
    Rmin = 0.0; Rguess = Inf ;
    prob = ODEProblem(TOV_equations!, u0, (Rmin, Rguess), par)
    TOV = init(prob, Tsit5(), dtmax=0.05, force_dtmin=true)
    R = 0.0
    while true
        T_Old = TOV.t #old radius
        step!(TOV)
        if TOV.u[1] < 0.0 #find zero
            f(r) = TOV.sol(r)[1] #density as function of radius
            R = find_zero(f, (T_Old, TOV.t), Order1())
            break
        end
    end
    return TOV.sol, R
end

make_TOV (generic function with 1 method)

In [5]:
function make_TOV_sequence(ρc, dP, par)
    mass = []
    radius = []
    for ρ₀ in ρc
        TOV,R = make_TOV(ρ₀, dP, par)
        push!(mass, TOV(R)[2])
        push!(radius,R)
    end
    return mass, radius
end

make_TOV_sequence (generic function with 1 method)

In [6]:
γ = 2; k = 123
P(ρ) = 123*ρ^2
dP(ρ) = 2*P(ρ)/ρ

print(P(1e-6),"\n", integrate_EOS(1e-6, ideal_gas_der, [123,2]))

1.23e-10
1.2299999999999996e-10

In [7]:
TOV, R = make_TOV(1e-3, ideal_gas_der, [123,2])
@time TOV, R = make_TOV(1e-3, ideal_gas_der, [123,2])
print(R)

  0.001008 seconds (3.04 k allocations: 271.320 KiB)
11.111051119437711

In [8]:
ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
M,R = make_TOV_sequence(ρ₀, ideal_gas_der, [123,2])


@time begin
    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
    M,R = make_TOV_sequence(ρ₀, ideal_gas_der, [123,2])
    plotlyjs()
    plot(R, M, xlabel = "Radius (km)", ylabel = "Mass (M⊙)", label = "Mat", title = "Neutron Stars K = 123, γ = 2", linewidth = 2)
end

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15809554817905758979\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8686621916339948659\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15809554817905758979\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

104.721070 seconds (110.27 M allocations: 4.014 GiB, 2.52% gc time)


In [9]:
plot(R*1.482, M, xlabel = "Radius (km)", ylabel = "Mass (M⊙)", label = "Summer", title = "Neutron Stars K = 123, γ = 2", linewidth = 2)

In [10]:
ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
x_interval = [0.0 2.683985562445732e-10; 2.683985562445732e-10 0.001387090243627891; 0.001387090243627891 0.0014224520180366636; 0.0014224520180366636 0.003935205491747961; 0.003935205491747961 0.01783099866626234];
y_interval = [0.0 0.0003043663708371769; 0.0003043663708371769 0.34006522767154; 0.34006522767154 0.33222981114647837; 0.33222981114647837 0.500866829421404; 0.500866829421404 0.6223683933290903];
M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


@time begin
    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot!(R.*1.482, M, label = "DNS(CMF)", xlabel = "Radius (km)", ylabel = "Mass (M⊙)", title = "Mass-Radius Relation", linewidth = 2)
end

  5.558278 seconds (5.70 M allocations: 247.778 MiB, 1.43% gc time, 5.48% compilation time)


In [11]:
ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
x_interval = [0.0 2.332827999180845e-13; 2.332827999180845e-13 0.0017014806718712407; 0.0017014806718712407 0.002367086328603055; 0.002367086328603055 0.002721945355829845; 0.002721945355829845 0.005571329734378488];
y_interval = [0.0 1.1297422315066974e-5; 1.1297422315066974e-5 0.5844155844155962; 0.5844155844155962 0.5025979497261605; 0.5025979497261605 0.5696797857737601; 0.5696797857737601 0.6892763157894719];
M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


@time begin
    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot!(R.*1.482, M, label = "BHK(QHC18)", xlabel = "Radius (km)", ylabel = "Mass (M⊙)", title = "Mass-Radius Relation", linewidth = 2)
end

  0.132574 seconds (304.46 k allocations: 25.200 MiB, 28.57% gc time)


In [12]:
ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);

x_interval = [0.0 2.121710214285006e-13; 2.121710214285006e-13 0.0014594123379490006; 0.0014594123379490006 0.0023525678858535635; 0.0023525678858535635 0.0026531025371383686; 0.0026531025371383686 0.007690387374332126]; 
y_interval = [0.0 1.096376754968612e-5; 1.096376754968612e-5 0.3739156386788699; 0.3739156386788699 0.4155102040816361; 0.4155102040816361 0.5193484129953385; 0.5193484129953385 0.6934721512519161];

M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


@time begin
    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);

    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot(R.*1.48, M, label = "Sequence 1", xlabel = "Radius (km)", ylabel = "Mass (M⊙)", title = "BFH(QHC19-A)", linewidth = 1.5, xlims = (13.53715, 19.72578))
end

  0.306354 seconds (476.47 k allocations: 32.886 MiB, 11.65% gc time, 58.29% compilation time)


In [13]:
ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
x_interval = [0.0 2.121710214285006e-13; 2.121710214285006e-13 0.0013445117079347335; 0.0013445117079347335 0.0021208038940020357; 0.0021208038940020357 0.00239146172602243; 0.00239146172602243 0.0068244762759529015]; 
y_interval = [0.0 1.096376754968612e-5; 1.096376754968612e-5 0.35212840897655373; 0.35212840897655373 0.3903714935557225; 0.3903714935557225 0.42266428252122956; 0.42266428252122956 0.6789121169068395];

M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


@time begin
    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot!(R.*1.48, M, linewidth = 1.5, label = "Sequence 2")
end

  0.186527 seconds (451.42 k allocations: 32.192 MiB, 41.28% compilation time)


In [14]:
ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
x_interval = [0.0 2.121710214285006e-13; 2.121710214285006e-13 0.0011982477779371706; 0.0011982477779371706 0.001901347385228805; 0.001901347385228805 0.0021588751186274427; 0.0021588751186274427 0.00600890347609279]; 
y_interval = [0.0 1.096376754968612e-5; 1.096376754968612e-5 0.3051356308523976; 0.3051356308523976 0.3842887473460706; 0.3842887473460706 0.3929001203369467; 0.3929001203369467 0.6623894759542822];

M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


@time begin
    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot!(R.*1.48, M, linewidth = 1.5, label = "Sequence 3")
end

  0.089478 seconds (282.44 k allocations: 23.405 MiB)


In [15]:
ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
x_interval = [0.0 2.121710214285006e-13; 2.121710214285006e-13 0.0010727599415600536; 0.0010727599415600536 0.001700692692771716; 0.001700692692771716 0.0019366938762646454; 0.0019366938762646454 0.005619849619591004]; 
y_interval = [0.0 1.096376754968612e-5; 1.096376754968612e-5 0.2547646587774711; 0.2547646587774711 0.3853417266187098; 0.3853417266187098 0.38445445767621794; 0.38445445767621794 0.653305141330959];

M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


@time begin
    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot!(R.*1.48, M, linewidth = 1.5, label = "Sequence 4")
end

  0.090674 seconds (272.19 k allocations: 22.491 MiB)


In [16]:
ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);

x_interval = [0.0 2.332827999180845e-13; 2.332827999180845e-13 0.0017014806718712407; 0.0017014806718712407 0.002367086328603055; 0.002367086328603055 0.002721945355829845; 0.002721945355829845 0.005571329734378488]; 
y_interval = [0.0 1.1297422315066974e-5; 1.1297422315066974e-5 0.5844155844155962; 0.5844155844155962 0.5025979497261605; 0.5025979497261605 0.5696797857737601; 0.5696797857737601 0.6892763157894719];

M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);

    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot(R, M, label = "Sequence 1", xlabel = "Radius (km)", ylabel = "Mass (M⊙)", title = "BHK(QHC18)", linewidth = 1.5) #xlims = (13.53715, 19.72578))

############################################################################################

ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
x_interval = [0.0 2.332827999180845e-13; 2.332827999180845e-13 0.0015369402449572204; 0.0015369402449572204 0.0021337608104041172; 0.0021337608104041172 0.002449985770791554; 0.002449985770791554 0.0050606153685544214]; 
y_interval = [0.0 1.1297422315066974e-5; 1.1297422315066974e-5 0.511291632818849; 0.511291632818849 0.5194396348182108; 0.5194396348182108 0.506196640044079; 0.506196640044079 0.6764766660926478];

M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot!(R, M, linewidth = 1.5, label = "Sequence 2")

############################################################################################

ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
x_interval = [0.0 2.332827999180845e-13; 2.332827999180845e-13 0.0013820980223450477; 0.0013820980223450477 0.001915539668131241; 0.001915539668131241 0.002208413892565718; 0.002208413892565718 0.0045733960571997115]; 
y_interval = [0.0 1.1297422315066974e-5; 1.1297422315066974e-5 0.39355391736670875; 0.39355391736670875 0.5633960156395615; 0.5633960156395615 0.5099277978339398; 0.5099277978339398 0.6621997471554996];

M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot!(R, M, linewidth = 1.5, label = "Sequence 3")

############################################################################################

ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
x_interval = [0.0 2.332827999180845e-13; 2.332827999180845e-13 0.0012318162648510052; 0.0012318162648510052 0.001728644303027397; 0.001728644303027397 0.001978768496534892; 0.001978768496534892 0.004109296572171727]; 
y_interval = [0.0 1.1297422315066974e-5; 1.1297422315066974e-5 0.2915869442881261; 0.2915869442881261 0.587415540540544; 0.587415540540544 0.5491339696005639; 0.5491339696005639 0.6458839150227601];

M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])


    ρ₀ = collect(1.0e-5:1.0e-4:1.0e-2);
    M,R = make_TOV_sequence(ρ₀, piece_wise_linear_csound, [x_interval, y_interval])
    plotlyjs()
    plot!(R, M, linewidth = 1.5, label = "Sequence 4")


In [17]:
k = 123.0; γ = 2.0
P(ρ) = k .*ρ .^γ
dP(ρ) = γ .*P(ρ) ./ρ

TOV, R = make_TOV(1.0e-2, P, dP)

#plot(TOV, vars = 1) #(RADIUS, DENSITY)

#plot(ρ₀ * 6.17e17, M, xlabel = "Central Density (g/cm³)", ylabel = "Mass (M⊙)", label = "Mat", title = "Neutron Stars K = 123, γ = 2", linewidth = 2)

LoadError: MethodError: no method matching P(::Float64, ::typeof(dP))
[0mClosest candidates are:
[0m  P(::Any) at In[17]:2